In [1]:
import os
import cv2
import warnings
import numpy as np 
import pandas as pd
import mediapipe as mp

from tqdm import tqdm
warnings.filterwarnings('ignore')

In [2]:
base_dir = "D:/sl_dataset/"

video_src = os.path.join(base_dir, "source")
keypoints_src = os.path.join(base_dir, "keypoints")

# mediapipe pose model init / mediapipe pose model 선언
mp_pose = mp.solutions.pose
pose_model = mp_pose.Pose(static_image_mode=True)   

In [3]:
def get_max_length(keypoints_list):
    max_length = 0  # val of max keypoints length / 키포인트 최대 길이값
    for keypoints_data in keypoints_list:
        current_length = keypoints_data.shape[0]    # keypoints length / 현재 keypoints의 길이값
        if current_length > max_length:
            max_length = current_length
    print(f"Max length : {max_length}")    
    return int(max_length)

In [ ]:
def extract_pose_keypoints(video_src, pose_model):
    pose_keypoints_list = []
    filename_list = []

    for video_file in tqdm(os.listdir(video_src)):
        video_path = os.path.join(video_src, video_file)  # generate file path / 파일 경로 생성
        filename = os.path.splitext(video_file)[0]  # filename remove extension / 파일 이름(확장자 제거)
        cap = cv2.VideoCapture(video_path)  # video load / 영상 읽어오기
        pose_keypoints = []  # list of pose keypoints / pose 키포인트를 저장할 리스트

        while cap.isOpened():
            ret, frame = cap.read()  # opened, frame read from video / 읽기 여부 프레임 읽기
            if not ret:
                break

            results = pose_model.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))  # pose process / pose 키포인트 추출 프로세스
            if results.pose_landmarks:
                keypoints = [[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]  # x, y, z
                pose_keypoints.append(keypoints)
        cap.release()

        keypoints_array = np.array(pose_keypoints)  # list to npy / 리스트 => numpy ndarray 변환
        pose_keypoints_list.append(keypoints_array)  # list of extracted keypoints data / 추출된 pose keypoints를 저장할 리스트
        filename_list.append(filename)  # filename added to list / 파일 이름을 리스트에 추가

    match_df = pd.DataFrame({
        'filename': filename_list,  # Filename list / 파일 이름 리스트
        'pose_keypoints': pose_keypoints_list  # Pose keypoints list / 포즈 키포인트 리스트
    })

    return match_df  # Return DataFrame / DataFrame 반환

match_df = extract_pose_keypoints(video_src, pose_model)

In [ ]:
max_length = get_max_length(match_df['pose_keypoints'])

for i, keypoints_data in enumerate(tqdm(match_df['pose_keypoints'])):
    filename = match_df['filename'][i]  # corrected indexing / 인덱싱 수정
    current_length = keypoints_data.shape[0]

    if max_length > current_length:
        padding = np.zeros((max_length - current_length, 33, 3))
        padded_keypoints = np.vstack((keypoints_data, padding))
    else:
        padded_keypoints = keypoints_data

    np.save(keypoints_src + f"/{filename}_padded.npy", padded_keypoints)
    print(f"{filename} - {padded_keypoints.shape}")

Max length : 165


100%|██████████| 15/15 [00:00<00:00, 2143.23it/s]

KETI_SL_0000000001 - (165, 33, 3)
KETI_SL_0000000002 - (165, 33, 3)
KETI_SL_0000000003 - (165, 33, 3)
KETI_SL_0000000004 - (165, 33, 3)
KETI_SL_0000000005 - (165, 33, 3)
KETI_SL_0000000006 - (165, 33, 3)
KETI_SL_0000000007 - (165, 33, 3)
KETI_SL_0000000008 - (165, 33, 3)
KETI_SL_0000000009 - (165, 33, 3)
KETI_SL_0000000010 - (165, 33, 3)
KETI_SL_0000000011 - (165, 33, 3)
KETI_SL_0000000012 - (165, 33, 3)
KETI_SL_0000000013 - (165, 33, 3)
KETI_SL_0000000014 - (165, 33, 3)
KETI_SL_0000000015 - (165, 33, 3)
